## Monthly Accounting

In [1]:
# General Libraries

import os
from bs4 import BeautifulSoup
import requests
import pymongo
import re
import json
from datetime import datetime
import time

# Dataframe manipulation
from glob import glob
import pandas as pd

# Database
import psycopg2
from config import config
# from sqlalchemy import create_engine
import sqlalchemy as sa

# Cryptography Library
from cryptography.fernet import Fernet

In [2]:
# Load Script Parameters

# Code References
    # https://www.scaler.com/topics/how-to-open-json-file-in-python/
    # https://bobbyhadz.com/blog/python-assign-dictionary-value-to-variable#assign-dictionary-key-value-pairs-to-variables-in-python

# Open json file with the parameters
with open("Monthly-Accounting-Params.json", "r") as file:
    jsonData = json.load(file)
# print("Datatype of variable: ", type(jsonData))

# Initiate variables
locals().update(jsonData['crypto'])
locals().update(jsonData['toggl'])
locals().update(jsonData['mail'])

# Decode passwords
key = bytes(pwd_key, 'utf-8')
fernet = Fernet(key)

mail_password = fernet.decrypt(bytes(mail_password, 'utf-8')).decode()

print(nd_mail_body)

Olá Elaine,

Em anexo a Nota de Débito do Mailchimp deste mês.

Oscar


In [3]:
# Create destination folder and/or mount the path

today = datetime.today()
current_year = today.year
current_month = today.month

# Processing year and month
month = '12' if current_month == 1 else str(current_month - 1).zfill(2)
year = str(current_year) if current_month > 1 else str(current_year - 1)
# print(month)
# print(year)

dest_folder = toggl_folder + year + '\\' + month
print(dest_folder)

if not os.path.isdir(dest_folder):
   os.mkdir(dest_folder)


D:\Primeway\Primeway\Administração\Planejamento Diário\Toggl\2023\07


In [9]:
# Connect to Google Driver
# Source: # https://stackoverflow.com/questions/66209119/automation-google-login-with-python-and-selenium-shows-this-browser-or-app-may

import undetected_chromedriver as uc

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Options
options = uc.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# prefs = {'download.default_directory' : 'D:\\Primeway\\Primeway\\Administração\\Planejamento Diário\\Toggl'}
prefs = {'download.default_directory' : dest_folder}
options.add_experimental_option('prefs', prefs)


driver = uc.Chrome(use_subprocess=True, options=options)
driver.get("https://track.toggl.com/")

In [10]:
# Get Gmail Account and Password

# email = input("Google Account:")
password = input("Google Password:")
print('Ok')

Ok


In [13]:
# Google Login

try:

   # driver.find_element(By.CSS_SELECTOR, '.css-154ozm6-SocialLogin').click()
   driver.find_element(By.CSS_SELECTOR, 'a.css-1dct05').click()

   email_field = WebDriverWait(driver,20).until(
      EC.presence_of_element_located((By.ID, 'identifierId'))
   )
   email_field.send_keys(mail_sender)
   email_field.send_keys(Keys.RETURN)
   
   driver.implicitly_wait(10)
   
   password_field = WebDriverWait(driver, 20).until(
      EC.presence_of_element_located((By.CSS_SELECTOR, 'input[name="Passwd"]'))
   )
   password_field.send_keys(password)
   password_field.send_keys(Keys.RETURN)
   
except Exception as error:
   print(error)

In [21]:
# Go to Previous Month Report Page and Download CSV File

try:

   driver.get('https://track.toggl.com/reports/detailed/1891953/period/prevMonth')

   # Detailed report - Click the download CSV link
   elem = WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.CSS_SELECTOR, '.css-4jnc92-Button-button-button-bodyText-PlainButton-EnhancedPlainButton'))
   )
   elem.click()
   elem = WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.CSS_SELECTOR, '.css-5drhz0-Container [data-heap-id="csv"]'))
   )
   elem.click()

except Exception as error:
   print(error)


In [4]:
# Verify if all Toggl task has an associated project

# print(dest_folder + "\\*.csv")

tasks = glob(dest_folder + "\\*.csv")[0]
df = pd.read_csv(tasks)

pd.options.display.max_rows = 999

# df

filtered_df = df[df['Project'].isnull()]

if filtered_df.size == 0:
   print("All tasks have an associated project!")
else:
   filtered_df

All tasks have an associated project!


In [ ]:
# a = df.columns.values
# # print(a)
# print(*a,sep='\n')

In [30]:
# Download PDF Files

def download_pdf():
   elem = WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.CSS_SELECTOR, '.css-4jnc92-Button-button-button-bodyText-PlainButton-EnhancedPlainButton'))
   )
   elem.click()
   elem = WebDriverWait(driver, 10).until(
      EC.presence_of_element_located((By.CSS_SELECTOR, '.css-5drhz0-Container [data-heap-id="pdf"]'))
   )
   elem.click()


try:

   # Detailed report - Click the download PDF link
   driver.get('https://track.toggl.com/reports/detailed/1891953/period/prevMonth')
   WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'reports-header')))
   download_pdf()

   time.sleep(10)

   # Sumemary report - Click the download PDF link
   driver.get('https://track.toggl.com/reports/summary/1891953/period/prevMonth')
   WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'reports-header')))
   download_pdf()

except Exception as error:
   print(error)


In [53]:
# Connect to PostgreSQL Database

# read connection parameters
params = config()

# connect to the PostgreSQL server
conn = psycopg2.connect(**params)
conn.autocommit = True
# create a cursor
cursor = conn.cursor()
print("psycopg2 ok!")

# Connect to database using SQLAlchemy
# Source: 
# https://www.geeksforgeeks.org/how-to-write-pandas-dataframe-to-postgresql-table/
# https://stackoverflow.com/questions/57761954/delete-rows-from-sql-server-bases-on-content-in-dataframe

conn_string = "postgresql+psycopg2://{}:{}@{}/{}".format(params['user'], params['password'], params['host'], params['database'])
alchemyEngine = sa.create_engine(conn_string)
alchemyConnection = alchemyEngine.connect()
print("SQLAlchemy ok!")


psycopg2 ok!
SQLAlchemy ok!


In [ ]:
# Save Toggl tasks into the PostgreSQL database

# Delete previous entries from the same month, if it exists
# select "Client", "Project", "Description", EXTRACT('Day' FROM "Start date"), EXTRACT('Year' FROM "Start date")  from time_entries
# Where EXTRACT('Day' FROM "Start date") = 2
sql = 'DELETE FROM time_entries WHERE EXTRACT("Year" FROM "Start date") = ' + year + ' AND EXTRACT("Month" FROM "Start date") = ' + month
cursor.execute(sql)

# Insert entries into time_entries table
df.to_sql('time_entries', alchemyConnection, if_exists='append', index=False)

In [54]:
# Read Toggls tasks from the PostgreSQL database

pd.set_option('display.max_rows', 500)

sql = """
   SELECT "Client", 
          "Project", 
          "Description", 
          EXTRACT("Day" FROM "Start date")::Integer AS "Day",
          "Duration", 
          ROUND((EXTRACT("hour" FROM "Duration")*60 + EXTRACT("minute" FROM "Duration") + 1)/60, 2) AS "Hours" 
   FROM time_entries 
   WHERE EXTRACT("Year" FROM "Start date") = '{}' AND EXTRACT("Month" FROM "Start date") = '{}'
"""
sql = sql.format(year, month)
df2 = pd.read_sql(sql, alchemyConnection)

# Close connections
conn.close()
alchemyConnection.close()

df2.head(10)
# df2[df2['Client'] == 'PNST' ]
# df2

,Client,Project,Description,Day,Duration,Hours
0,Primeway,Treinamento,Economics study,1,01:02:46,1.05
1,Primeway,Treinamento,Statistics Study,1,01:00:38,1.02
2,Primeway,Treinamento,Statistics Study,2,00:00:30,0.02
3,Primeway,Treinamento,Statistics Study,2,01:00:00,1.02
4,Primeway,Treinamento,Economics study,2,00:19:15,0.33
5,Primeway,Treinamento,Economics study,2,00:09:41,0.17
6,Primeway,Treinamento,Economics study,2,00:32:20,0.55
7,Primeway,Serviços Gerais,Financial Management,2,01:01:18,1.03
8,Primeway,Serviços Gerais,Workstation preparation,2,01:39:05,1.67
9,Primeway,Serviços Gerais,Daily Organization,3,00:17:16,0.30


In [119]:
# Get Grouped task by Client/Project/Day

# reference: https://stackoverflow.com/questions/10373660/converting-a-pandas-groupby-output-from-series-to-dataframe

df_grouped = df2.groupby(["Client", "Project", "Day", "Description"])[["Hours"]].sum()
# print(df_grouped)

df_grouped = df_grouped.reset_index()
df_grouped[['Day',  "Description", "Hours"]].head()

,Day,Description,Hours
0,27,Alteração ano home-page,0.17
1,7,Montagem Artigo,0.30
2,7,Publicação de Vídeo,0.70
3,13,Atualização de plugins,0.18
4,13,Erro no carregamento de imagens,0.35


In [143]:
# Show PNST tasks

df_grouped.index=[''] * len(df_grouped)

# PNST
print('Atividades PNST:')
df_pnst = df_grouped[(df_grouped['Client'] == 'PNST') & (df_grouped['Project'] == 'Manutenção Site')]
df_pnst[['Day',  "Description", "Hours"]]

Atividades PNST:


,Day,Description,Hours
,7,Montagem Artigo,0.30
,7,Publicação de Vídeo,0.70
,13,Atualização de plugins,0.18
,13,Erro no carregamento de imagens,0.35
,13,Publicação de artigos,0.53
,13,Retirada de profissional,0.55
,19,Montagem newsletter,1.04
,21,Inclusão de artigo,0.27
,25,Atendimento Tais,0.12
,26,Manutenções de segurança do site,1.03


In [145]:
# Show Swedcham tasks

print('Atividades Swedcham:')
df_pnst = df_grouped[(df_grouped['Client'] == 'Swedcham') & (df_grouped['Project'] == 'Manutenção Site')]
df_pnst[['Day',  "Description", "Hours"]]


Atividades Swedcham:


,Day,Description,Hours
,3,Publicação de artigo,0.15
,4,Alteração rodapé do site,0.13
,10,Contato Hélio,0.07
,11,Análise venda de dados Swedcham,0.53
,11,Contato Mailchimp,0.50
,12,Atualização do tema do site,0.20
,12,Atualizações PHP e Plugins,0.50
,12,Contato Mailchimp,0.38
,13,Atendimento Jonas,0.47
,14,Contato Hacker,0.27
